In [35]:
%%capture
import tensorflow as tf
from tensorflow import keras
from scipy.io import loadmat
from tensorflow.keras import regularizers
import os
import time
import numpy as np
import gc
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from sklearn.metrics import r2_score
from sklearn import preprocessing
import gc
import h5py
import math
from IPython.display import HTML, display
import tabulate

In [36]:
def scale(x):
    return (x -np.mean(x)) / np.std(x)

In [37]:
def preprocess(x): 
    return x[:,:,:int(0.75*x.shape[2])]

In [38]:
os.chdir("/home/llin2948/dscaptstone")
path = os.getcwd()
path

'/home/llin2948/dscaptstone'

In [39]:
os.chdir(path+"/Saved_Models_2Sets")
##### Load Model for tau_12
json_file = open('model2_Heat_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_12 = model_from_json(loaded_model_json)

# load weights into new model
loaded_model_12.load_weights("model2_Heat_1.h5")
print("Loaded model 12 from disk")

# evaluate loaded model on test data 
# Define X_test & Y_test data first
loaded_model_12.compile(optimizer = tf.train.AdamOptimizer(), loss = "mse")

#### Load Model for tau_13
json_file = open('model2_Heat_2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_13 = model_from_json(loaded_model_json)

# load weights into new model
loaded_model_13.load_weights("model2_Heat_2.h5")
print("Loaded model 13 from disk")

# evaluate loaded model on test data 
# Define X_test & Y_test data first
loaded_model_13.compile(optimizer = tf.train.AdamOptimizer(), loss = "mse")

#### Load Model for tau_23
json_file = open('model2_Heat_3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_23 = model_from_json(loaded_model_json)

# load weights into new model
loaded_model_23.load_weights("model2_Heat_3.h5")
print("Loaded model 23 from disk")

# evaluate loaded model on test data 
# Define X_test & Y_test data first
loaded_model_23.compile(optimizer = tf.train.AdamOptimizer(), loss = "mse")

Loaded model 12 from disk
Loaded model 13 from disk
Loaded model 23 from disk


In [42]:
import math
from IPython.display import HTML, display
import tabulate
out = [["Data Source", "Result", "Heat_1", "Heat_2", "Heat_3"]]
dirs = [("Re15sh2_DNScoarse8",8),("Re15sh5_DNScoarse8",8),("Re15sh20_DNScoarse8",8),("Re15sh20_DNScoarse16",16),("Re15sh20_DNScoarse24",24)]
for (dataDir, coarse) in dirs:
    dirName = path + "/Data/" + dataDir
    os.chdir(dirName)
    print("Processing " + dirName)
    u = preprocess(scale(loadmat('u_F_xyz_T1_DNS'+str(coarse)+'.mat')["u_F"]))
    v = preprocess(scale(loadmat('v_F_xyz_T1_DNS'+str(coarse)+'.mat')["v_F"]))
    w = preprocess(scale(loadmat('w_F_xyz_T1_DNS'+str(coarse)+'.mat')["w_F"]))
    tau_12 = preprocess(scale(loadmat('Heat1_F_xyz_T1_DNS'+str(coarse)+'.mat')["Heat1"]))
    tau_13 = preprocess(scale(loadmat('Heat2_F_xyz_T1_DNS'+str(coarse)+'.mat')["Heat2"]))
    tau_23 = preprocess(scale(loadmat('Heat3_F_xyz_T1_DNS'+str(coarse)+'.mat')["Heat3"]))

    x = np.array([u, v, w])
    x = np.transpose(x, [1, 2, 3, 0])
    x = np.pad(x, ((3,3), (3,3), (3,3), (0,0)), 'constant', constant_values = 0)

    sample = []
    size = 3
    y_tau_12 = []
    y_tau_13 = []
    y_tau_23 = []
    for i in tqdm_notebook(range(size, x.shape[0] - size)):
        for j in range(size, x.shape[1] - size):
            for k in range(size, x.shape[2] - size):
                sample.append(x[i - size: i + size + 1, j - size: j + size + 1, k - size: k + size + 1, :])
                y_tau_12.append(tau_12[i - size][j - size][k - size])
                y_tau_13.append(tau_13[i - size][j - size][k - size])
                y_tau_23.append(tau_23[i - size][j - size][k - size])
            
    x = np.array(sample)
    y_tau_12 = np.array(y_tau_12)
    y_tau_13 = np.array(y_tau_13)
    y_tau_23 = np.array(y_tau_23)

    print("predicting tau_12")
    y_pred_12 = loaded_model_12.predict(x).flatten()

    print("predicting tau_13")
    y_pred_13 = loaded_model_13.predict(x).flatten()

    print("predicting tau_23")
    y_pred_23 = loaded_model_23.predict(x).flatten()
    
    out.append([dataDir, 'Corr', str(np.round(np.corrcoef(y_pred_12, y_tau_12)[0, 1], 4)), str(np.round(np.corrcoef(y_pred_13, y_tau_13)[0, 1], 4)), str(np.round(np.corrcoef(y_pred_23, y_tau_23)[0, 1], 4))])
    out.append(['', 'R^2', str(np.round(r2_score(y_tau_12, y_pred_12), 4)), str(np.round(r2_score(y_tau_13, y_pred_13), 4)), str(np.round(r2_score(y_tau_23, y_pred_23), 4))])
    
display(HTML(tabulate.tabulate(out, tablefmt = 'html')))

Processing /home/llin2948/dscaptstone/Data/Re15sh2_DNScoarse8



predicting tau_12
predicting tau_13
predicting tau_23
Processing /home/llin2948/dscaptstone/Data/Re15sh5_DNScoarse8



predicting tau_12
predicting tau_13
predicting tau_23
Processing /home/llin2948/dscaptstone/Data/Re15sh20_DNScoarse8



predicting tau_12
predicting tau_13
predicting tau_23
Processing /home/llin2948/dscaptstone/Data/Re15sh20_DNScoarse16



predicting tau_12
predicting tau_13
predicting tau_23
Processing /home/llin2948/dscaptstone/Data/Re15sh20_DNScoarse24



predicting tau_12
predicting tau_13
predicting tau_23


Data Source,Result,Heat_1,Heat_2,Heat_3
Re15sh2_DNScoarse8,Corr,0.7382,0.7492,0.8694
,R^2,0.4908,0.5054,0.6947
Re15sh5_DNScoarse8,Corr,0.5807,0.6226,0.7684
,R^2,0.322,0.3696,0.5412
Re15sh20_DNScoarse8,Corr,0.8849,0.7688,0.8471
,R^2,0.7097,0.5392,0.6594
Re15sh20_DNScoarse16,Corr,0.7575,0.643,0.8034
,R^2,0.5561,0.3638,0.4488
Re15sh20_DNScoarse24,Corr,0.6707,0.576,0.7886
,R^2,0.2597,-0.0928,-0.6867


In [40]:
import math
from IPython.display import HTML, display
import tabulate
out = [["Data Source", "Result", "Heat_1", "Heat_2", "Heat_3"]]
dirs = [("Re15sh20_DNScoarse4",4)]
for (dataDir, coarse) in dirs:
    dirName = path + "/Data/" + dataDir
    os.chdir(dirName)
    print("Processing " + dirName)
    u = preprocess(scale(loadmat('u_F_xyz_T1_DNS'+str(coarse)+'.mat')["u_F"]))
    v = preprocess(scale(loadmat('v_F_xyz_T1_DNS'+str(coarse)+'.mat')["v_F"]))
    w = preprocess(scale(loadmat('w_F_xyz_T1_DNS'+str(coarse)+'.mat')["w_F"]))
    tau_12 = preprocess(scale(loadmat('Heat1_F_xyz_T1_DNS'+str(coarse)+'.mat')["Heat1"]))
    tau_13 = preprocess(scale(loadmat('Heat2_F_xyz_T1_DNS'+str(coarse)+'.mat')["Heat2"]))
    tau_23 = preprocess(scale(loadmat('Heat3_F_xyz_T1_DNS'+str(coarse)+'.mat')["Heat3"]))

    x = np.array([u, v, w])
    x = np.transpose(x, [1, 2, 3, 0])
    x = np.pad(x, ((3,3), (3,3), (3,3), (0,0)), 'constant', constant_values = 0)

    size = 3
    y_pred_12 = np.array([])
    y_pred_13 = np.array([])
    y_pred_23 = np.array([])
    y_true_12 = np.array([])
    y_true_13 = np.array([])
    y_true_23 = np.array([])
    u_idx = 0
    v_idx = 0
    z_idx = 0
    for u_idx in range(0, math.ceil(x.shape[0]/151)):
        for v_idx in range(0, math.ceil(x.shape[1]/101)):
            for z_idx in range(0, math.ceil(x.shape[2]/60)):
                u_lowerbound = u_idx*151+size
                u_upperbound = x.shape[0] - size if (u_idx+1)*151 >= x.shape[0] else (u_idx+1)*151 - size
                v_lowerbound = v_idx*101+size
                v_upperbound = x.shape[1] - size if (v_idx+1)*101 >= x.shape[1] else (v_idx+1)*101 - size
                z_lowerbound = z_idx*60+size
                z_upperbound = x.shape[2] - size if (z_idx+1)*60 >= x.shape[2] else (z_idx+1)*60 - size
                print(str(u_lowerbound) + ' ' + str(u_upperbound))
                print(str(v_lowerbound) + ' ' + str(v_upperbound))
                print(str(z_lowerbound) + ' ' + str(z_upperbound))
                sample = []
                y_tau_12 = []
                y_tau_13 = []
                y_tau_23 = []
                for i in tqdm_notebook(range(u_lowerbound, u_upperbound)):
                    for j in range(v_lowerbound, v_upperbound):
                        for k in range(z_lowerbound, z_upperbound):
                            sample.append(x[i - size: i + size + 1, j - size: j + size + 1, k - size: k + size + 1, :])
                            y_tau_12.append(tau_12[i - size][j - size][k - size])
                            y_tau_13.append(tau_13[i - size][j - size][k - size])
                            y_tau_23.append(tau_23[i - size][j - size][k - size])

                x_part = np.stack(sample, axis = 0)
                y_tau_12 = np.array(y_tau_12)
                y_tau_13 = np.array(y_tau_13)
                y_tau_23 = np.array(y_tau_23)

                del sample
                gc.collect()

                y_pred_12 = np.append(y_pred_12, loaded_model_12.predict(x_part).flatten())
                y_true_12 = np.append(y_true_12, y_tau_12)

                y_pred_13 = np.append(y_pred_13, loaded_model_13.predict(x_part).flatten())
                y_true_13 = np.append(y_true_13, y_tau_13)

                y_pred_23 = np.append(y_pred_23, loaded_model_23.predict(x_part).flatten())
                y_true_23 = np.append(y_true_23, y_tau_23)

                del x_part
                del y_tau_12
                del y_tau_13
                del y_tau_23
                gc.collect()

    out.append([dataDir, 'Corr', str(np.round(np.corrcoef(y_pred_12, y_true_12)[0, 1], 4)), str(np.round(np.corrcoef(y_pred_13, y_true_13)[0, 1], 4)), str(np.round(np.corrcoef(y_pred_23, y_true_23)[0, 1], 4))])
    out.append(['', 'R^2', str(np.round(r2_score(y_true_12, y_pred_12), 4)), str(np.round(r2_score(y_true_13, y_pred_13), 4)), str(np.round(r2_score(y_true_23, y_pred_23), 4))])
    
display(HTML(tabulate.tabulate(out, tablefmt = 'html')))

Processing /home/llin2948/dscaptstone/Data/Re15sh20_DNScoarse4
3 148
3 98
3 57



3 148
3 98
63 117



3 148
104 199
3 57



3 148
104 199
63 117



154 299
3 98
3 57



154 299
3 98
63 117



154 299
104 199
3 57



154 299
104 199
63 117


Data Source,Result,Heat_1,Heat_2,Heat_3
Re15sh20_DNScoarse4,Corr,0.8088,0.5907,0.6794
,R^2,0.5005,0.2578,0.2842
